In [1]:
import pandas as pd
import tensorflow as tf
# import numpy as np

# tf.set_random_seed(1)
# np.random.seed(1)
print('1111')

def to_onehot(y):
    for i in range(len(y)):
        if y.iloc[i,0]==0:
            y.iloc[i,0] = 'y'
        else:
            y.iloc[i,0] = 'f'
    return pd.get_dummies(y,prefix=y.columns)

1111


In [2]:
# 参数，超参数
batch_size = 50
time_step = 40
input_size = 40
lr = 0.5
print('1111')

1111


In [3]:
# 准备数据
train_data_set = pd.read_csv('train.csv')                   # shape:(4000,1602)
train_data = train_data_set.iloc[0:4000,1:1601]             # shape:(4000,1600)
train_labels = to_onehot(train_data_set.iloc[0:4000,[1601]])            # shape:(4000,1)
print('1111')

1111


In [4]:
# 选择部分数据集
test_data_set = pd.read_csv('test.csv')                # shape:(3550,1601) 没有包含正确结果，和训练集不一样
test_data = test_data_set.iloc[0:1000,1:1601]           # shape:(1000,1600)
test_labels = to_onehot(
    pd.read_csv('sample_submit.csv').iloc[0:1000,[1]])          # shape:(1000,1) 一一对应
print('1111')

1111


In [5]:
# 创建模型
tf_x = tf.placeholder(tf.float32,[None,time_step*input_size])
x_img = tf.reshape(tf_x,[-1,time_step,input_size])
tf_y = tf.placeholder(tf.int32,[None,2])

## RNN
rnn_cell = tf.contrib.rnn.BasicLSTMCell(num_units = 50)
output_ , states = tf.nn.dynamic_rnn(
        rnn_cell,               #cell
        x_img,                  #input
        initial_state = None,   #初始化隐藏层的状态？表示不初始化？
        dtype = tf.float32,     #和上面那句必须同时出现
        time_major = False      #False: (batch, time_step, input); True: (time_step, batch, input)
)
output = tf.layers.dense(output_[:,-1,:],2) #对输出进行最后的修改

loss = tf.losses.softmax_cross_entropy(onehot_labels=tf_y,logits = output)
train_op = tf.train.AdamOptimizer(lr).minimize(loss)

# 计算精度
# 计算精度
accuracy = tf.metrics.accuracy(
        labels = tf.argmax(tf_y,axis = 1),
        predictions = tf.argmax(output,axis = 1)
)[1]
print('1111')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

1111


In [6]:
sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)
print('1111')

1111


In [9]:
for step in range(351):
    b_x = train_data.iloc[step*batch_size:(step+1)*batch_size]
    b_y = train_labels.iloc[step*batch_size:(step+1)*batch_size]
    _,loss_ = sess.run([train_op,loss],feed_dict = {tf_x:b_x,tf_y:b_y})
    if step % batch_size ==0:
        accuracy_= sess.run(accuracy,feed_dict = {tf_x:test_data,tf_y:test_labels})
        print('Step:', step,'| train loss: %.4f' % loss_, '| test accuracy: %.8f' % accuracy_)
    if step == 299:
        predictions_2 = tf.argmax(output,axis = 1)
        test_data_ = test_data
        predictions_2_ ,output_= sess.run([predictions_2,output], feed_dict = {tf_x: test_data_})
        print('predictions_2:', predictions_2_,'output:',output_)

Step: 0 | train loss: 3.4960 | test accuracy: 0.50222224
Step: 50 | train loss: 0.7562 | test accuracy: 0.50199997
Step: 100 | train loss: 0.0000 | test accuracy: 0.50181818
Step: 150 | train loss: 0.0000 | test accuracy: 0.50166667
Step: 200 | train loss: 0.0000 | test accuracy: 0.50153846
Step: 250 | train loss: 0.0000 | test accuracy: 0.50142854
predictions_2: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0